In [2]:
import ueye
import numpy as np
from matplotlib.pyplot import *
import os
from skimage.measure import label

# EM algorithms 
adapted from Engbert et al.

**Parameters to be tuned:**

- lambda

dataset 1: 4.4

dataset 2: 5.5

dataset 3: 4.0


In [7]:
# Cross Validation: take the cross validation set, split into 10folds. Use 90% for parameter tuning and 10 % to test.
lambdas = np.arange(1,11,.2) #lambda candidates


K = 10 #10fold cross validation
Ntrain = [950,950,330]
datasets = ['dataset1','dataset2','dataset3']
Sampfreq=[1000,1000,500]
min_sacc_dur = 6
min_distance = [10,5,10]
for i,dset in enumerate(datasets):
    
    ntrain = Ntrain[i]
    min_sacc_dist = min_distance[i]
    sampfreq = Sampfreq[i]
    fac = 1000/sampfreq
    datapath = '../data/' + dset
    print(datapath)
    X = np.loadtxt(datapath+'/'+dset+'_'+str(sampfreq)+'hz_X_train.csv',delimiter=',')
    Y = np.loadtxt(datapath+'/'+dset+'_'+str(sampfreq)+'hz_Y_train.csv',delimiter=',')
    L = np.loadtxt(datapath+'/'+dset+'_'+str(sampfreq)+'hz_Labels_train.csv',delimiter=',')
    L = (L==1).astype(float)
    # chose same subset as was used for network
    np.random.seed(1)
    randind = np.random.permutation(X.shape[0])
    X = X[randind[:ntrain],:]
    Y = Y[randind[:ntrain],:]
    L = L[randind[:ntrain],:]

    n_samples,n_time = X.shape
    
    np.random.seed(1)
    n_test = int(n_samples/K)
    Kappa = np.zeros(K)
    F1 = np.zeros(K)
    On = []
    Off = []
    indices = np.random.permutation(n_samples) #indices from training and test set
    # CROSS VALIDATION
    for i in range(K):
        ind_train = indices.copy()
        if i==K-1:
            ind_train = np.array(np.delete(ind_train,range(n_test*i,n_samples)))
            ind_test = indices[n_test*i:]
        else:
            ind_train = np.array(np.delete(ind_train,range(n_test*i,n_test*(i+1))))
            ind_test = indices[n_test*i:n_test*(i+1)]
            
        # training and test set
        Xtrain = X[ind_train,:].copy()
        Ytrain = Y[ind_train,:].copy()
        Xtest = X[ind_test,:].copy()
        Ytest = Y[ind_test,:].copy()
        Ltrain = L[ind_train,:].copy()  
        Ltest = L[ind_test,:].copy()
        kappa = np.zeros(len(lambdas))
        for j,lam in enumerate(lambdas):
            Sacc_out = ueye.functions.EM_saccade_detection(Xtrain,Ytrain,lambda_param=lam,
                                                          min_sacc_dist=min_sacc_dist,min_sacc_dur=min_sacc_dur,
                                                           sampfreq=sampfreq)
            kappa[j] = ueye.functions.binary_cohens_kappa(Ltrain.astype(float).flatten(),Sacc_out.astype(float).flatten())

        best_lam = lambdas[np.argmax(kappa)]
        print('best lambda:',best_lam)
        
        # TEST
        Sacc_out = ueye.functions.EM_saccade_detection(Xtest,Ytest,lambda_param=best_lam,
                                                      min_sacc_dist=5,min_sacc_dur=6,
                                                           sampfreq=sampfreq)
    
        Kappa[i] = ueye.functions.binary_cohens_kappa(Ltest.astype(float).flatten(),Sacc_out.astype(float).flatten()) 
        true_pos,false_pos,false_neg,on_distance,off_distance = ueye.functions.accuracy(Sacc_out.astype(float),Ltest.astype(float))
        F1[i] = (2 * true_pos)/(2 * true_pos + false_neg + false_pos)   
        On.append(on_distance)
        Off.append(off_distance)
    Performance = {
        'kappa': Kappa,
        'f1': F1,
        'on': On,
        'off': Off
        }
    np.save('summary/crossvalidation/em_'+dset,Performance)
        


data/dataset1


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/lib/function_base.py:4011: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/Users/marieschmidt/Dropbox/ueye/ueye/functions.py:465: RuntimeWarning: invalid value encountered in greater
  Sacc_out[i,:]=(np.divide(Vx[i,:],(lambda_param*np.sqrt(Eta_x[i])))**2+np.divide(Vy[i,:],(lambda_param*np.sqrt(Eta_y[i])))**2)>1


best lambda: 4.4
best lambda: 4.4
best lambda: 4.4
best lambda: 4.4
best lambda: 4.4
best lambda: 4.4
best lambda: 4.4
best lambda: 4.4
best lambda: 4.4
best lambda: 4.4
data/dataset2
best lambda: 5.6
best lambda: 5.2
best lambda: 5.6
best lambda: 5.4
best lambda: 5.2
best lambda: 5.4
best lambda: 5.2
best lambda: 5.6
best lambda: 5.6
best lambda: 5.4
data/dataset3
best lambda: 4.0
best lambda: 4.0
best lambda: 4.0
best lambda: 4.0
best lambda: 4.0
best lambda: 4.0
best lambda: 4.0
best lambda: 4.0
best lambda: 4.0
best lambda: 4.0
